In [42]:
import torch 
import einops
import math
import re
model = torch.load("/Users/khoi.ho/Downloads/Upscale_UL.pth", map_location=torch.device('mps'))

def convert(weight, bias, data, doswap=False):
    swap = [0,2,1,3]
    out_chan, in_chan, width, height = weight.shape
    for to in range(math.ceil(out_chan/4)):
        for ti in range(math.ceil(in_chan/4)):
            for w in range(width):
                for h in range(height):
                    for i in range(min(4, in_chan)):
                        for o in range(min(4, out_chan)):
                            o = swap[o] if doswap else o
                            data.append(float(weight[to*4+o, ti*4+i, w, h]))
        for o in range(min(4, out_chan)):
            o = swap[o] if doswap else o
            data.append(float(bias.data[to*4+o]))

layers = [i[:-7] for i in model.keys() if ".weight" in i]
data = []
for i in layers:
    convert(model[i+".weight"], model[i+".bias"], data)
len(data)

17412

In [ ]:
with open("tmp/Anime4K_Upscale_CNN_x2_UL.glsl") as f:
    text = f.read()
data_iter = iter(data)
def replace_match(match):
    return str(next(data_iter))

pattern = r'-?\d+(\.\d{2,})(e-?\d+)?'

a = re.sub(pattern, replace_match, text)
print(a)